In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from delta import *
from create_spark import CreateSparkSession

In [2]:
spark_obj = CreateSparkSession()
spark = spark_obj.session

23/09/26 00:19:18 WARN Utils: Your hostname, Wonseoks-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.35.161 instead (on interface en0)
23/09/26 00:19:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/wonseokchoi/.ivy2/cache
The jars for the packages stored in: /Users/wonseokchoi/.ivy2/jars
com.amazonaws#aws-java-sdk added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b2b1d829-134a-4fc9-b714-2bac3f5abe96;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/wonseokchoi/Documents/Installs/spark-3.4.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.amazonaws#aws-java-sdk;1.11.563 in central
	found com.amazonaws#aws-java-sdk-ioteventsdata;1.11.563 in central
	found com.amazonaws#aws-java-sdk-core;1.11.563 in central
	found commons-logging#commons-logging;1.1.3 in central
	found org.apache.httpcomponents#httpclient;4.5.5 in central
	found org.apache.httpcomponents#httpcore;4.4.9 in central
	found commons-codec#commons-codec;1.10 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found com.fasterxml.jackson.core#jackson-databind;2.6.7.2 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.6.0 in central
	found com.fasterxml.jackson.core#jackson-core;2.6.7 in central
	found com.fasterxml.jackson.dataformat#jackson-dataformat-cbor;2.6.7 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.11.563 in central
	found com.amazonaws#aws-java-sdk-iotevents;1.11.563 in central
	found com.amazonaws#aws-java-sdk-iotthingsgraph;1.11.563 in central
	found com.amazonaws#

In [3]:
spark.sql("optimize 's3://wonseokchoi-data-lake-project/lake/cassandra_replication/price/all_stocks/'")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [4]:
df = spark.read \
    .format("delta") \
    .load("s3://wonseokchoi-data-lake-project/lake/cassandra_replication/price/all_stocks/")
df.printSchema()

root
 |-- time_bucket: long (nullable = true)
 |-- symbol: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- open: double (nullable = true)
 |-- transactions: long (nullable = true)
 |-- updated_at: timestamp (nullable = true)
 |-- volume: long (nullable = true)
 |-- volume_weighted: long (nullable = true)



In [5]:
df1 = df.withColumn("timestamp", F.date_format(F.from_unixtime(F.col("timestamp")/1000), "yyyy-MM-dd HH:mm:ss"))
df_cleaned = df1.drop("updated_at", "time_bucket")
df_tsla = df_cleaned.filter(F.col("symbol") == "TSLA").orderBy(F.col("timestamp").asc())
df_tsla.printSchema()
df_tsla.show(10)

root
 |-- symbol: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- open: double (nullable = true)
 |-- transactions: long (nullable = true)
 |-- volume: long (nullable = true)
 |-- volume_weighted: long (nullable = true)



+------+-------------------+-------+-------+-------+-------+------------+------+---------------+
|symbol|          timestamp|  close|   high|    low|   open|transactions|volume|volume_weighted|
+------+-------------------+-------+-------+-------+-------+------------+------+---------------+
|  TSLA|2018-09-06 17:00:00|  18.83|  18.83|  18.83|  18.83|           5|  3015|             18|
|  TSLA|2018-09-06 17:29:00|18.9633|18.9633|18.9633|18.9633|           6|  6000|             18|
|  TSLA|2018-09-06 17:34:00| 18.992| 18.992| 18.976| 18.976|          13|  8850|             18|
|  TSLA|2018-09-06 17:41:00|  18.94|  18.94|  18.94|  18.94|           2|  2100|             18|
|  TSLA|2018-09-06 17:44:00|18.9667|18.9667|18.9667|18.9667|           1|  1500|             18|
|  TSLA|2018-09-06 17:45:00|   19.0|   19.0|   19.0|   19.0|          12| 17460|             18|
|  TSLA|2018-09-06 17:50:00|19.0533|19.0533|19.0527|19.0527|           5|  4740|             19|
|  TSLA|2018-09-06 17:51:00|19

In [6]:
from delta.tables import DeltaTable

df_tsla.coalesce(1).write.format("delta").mode("overwrite").save("s3://wonseokchoi-data-lake-project/lake/cleaned/price_cleaned/TSLA/")

deltaTable = DeltaTable.forPath(spark, "s3://wonseokchoi-data-lake-project/lake/cleaned/price_cleaned/TSLA/")
deltaTable.optimize()

In [7]:
DF = df_tsla.toPandas()
DF.head(10)
# DF.to_csv("../data/csv/tsla.csv", index=False)

,symbol,timestamp,close,high,low,open,transactions,volume,volume_weighted
0,TSLA,2018-09-06 17:00:00,18.8300,18.8300,18.8300,18.8300,5.0,3015.0,18.0
1,TSLA,2018-09-06 17:29:00,18.9633,18.9633,18.9633,18.9633,6.0,6000.0,18.0
2,TSLA,2018-09-06 17:34:00,18.9920,18.9920,18.9760,18.9760,13.0,8850.0,18.0
3,TSLA,2018-09-06 17:41:00,18.9400,18.9400,18.9400,18.9400,2.0,2100.0,18.0
4,TSLA,2018-09-06 17:44:00,18.9667,18.9667,18.9667,18.9667,1.0,1500.0,18.0
5,TSLA,2018-09-06 17:45:00,19.0000,19.0000,19.0000,19.0000,12.0,17460.0,18.0
6,TSLA,2018-09-06 17:50:00,19.0533,19.0533,19.0527,19.0527,5.0,4740.0,19.0
7,TSLA,2018-09-06 17:51:00,19.0333,19.0333,19.0333,19.0333,1.0,7500.0,19.0
8,TSLA,2018-09-06 17:54:00,19.0533,19.0533,19.0533,19.0533,10.0,13080.0,19.0
9,TSLA,2018-09-06 17:58:00,19.2413,19.2660,19.1313,19.1333,23.0,31950.0,19.0
